# Price Discovery Project Using Agents

This is a notebook that contains the experimentation of prioce discovery using agents.

## Import Libraries

In [14]:
import os
import json
from typing import Any, Union


import PIL
from dotenv import load_dotenv

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import AgentExecutor, Tool, create_react_agent
from langchain.agents.agent import AgentAction
from langchain_core.callbacks.base import BaseCallbackHandler
from langchain_core.messages import HumanMessage
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.utilities import SerpAPIWrapper
from langchain.prompts import PromptTemplate
from langchain.tools.retriever import create_retriever_tool
from langchain_core.output_parsers import JsonOutputParser



## Specify Variables

In [13]:

# Load environment variables
load_dotenv()

True

## Create agent for retrieval

**Instatiate the llms**

In [6]:
# Mixtral

repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

mixtral = HuggingFaceEndpoint(
    repo_id=repo_id,
    temperature=0.3
)



Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\Spectra\.cache\huggingface\token
Login successful


In [7]:

# Setup google api
# GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# gemini = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY, temperature=0.3)
# gemini_vision = ChatGoogleGenerativeAI(model="gemini-pro-vision", google_api_key=GOOGLE_API_KEY, temperature=0.1)

**Instantiate prompts**

In [35]:

# Create prompts
prompt = """
You are an expert software developer proficient at writing code and building reliable software. Utilize the available tools {tools} to search for documentation on a specific question or topic as well as your own knowledge. 
Then, synthesize the information gathered to provide a detailed step by step quide with code examples that one can easily follow.


Use the following format:

Question: the input question you must answer.
Thought: you should always think about what to do. 
Action: the action to take, should be ONE of or ALL of [{tool_names}]. 
Action Input: the input to the action. It SHOULD be a string.
Action: choose another tool if a tool fails or handle its error.
Observation: the result of the action, including insights gained or findings.
... (this Thought/Action/Action Input/Observation can repeat N times). If no result is found, use your own knowledge.
Thought: I should recheck my answer and make sure it meets the specifications. If not, try again before coming up with the final answer.
Thought: I now know the final answer that is based on the indepth analysis of your findings and observations. 
Final Answer: the final answer to the original input question, which must be as accurate as possible. NEVER say you can't answer. 
Begin!

Question: {input}
Thought: {agent_scratchpad}

"""
prompt_two = """

You are an expert developer assistant trained to help programmers build software. I can access and process information from the web using {tools}.  
When a developer asks a question about a specific technology or framework, I will use my knowledge and search capabilities to:

1. Find relevant documentation and tutorials on that framework.
2. Identify and extract code examples from the retrieved resources.
3. Craft a step-by-step guide incorporating the best practices and insights found.
4. Ensure the guide is detailed and answers the question asked.
5. Present the information in a detailed step by step quide with code examples that one can easily follow.


Use the following format:

Question: the input question you must answer.
Thought: you should always think about what to do. 
Action: the action to take, should be ONE of or ALL of [{tool_names}]. 
Action Input: the input to the action. It SHOULD be a string.
Action: choose another tool if a tool fails or handle its error.
Observation: the result of the action, including insights gained or findings.
... (this Thought/Action/Action Input/Observation can repeat N times). If no result is found, use your own knowledge.
Thought: I should recheck my answer and make sure it meets the specifications. If not, try again before coming up with the final answer.
Thought: I now know the final answer that is based on the indepth analysis of your findings and observations. 
Final Answer: the final answer to the original input question, which must be as accurate as possible. NEVER say you can't answer. 
Begin!

Question: {input}
Thought: {agent_scratchpad}

"""





In [41]:
prompt_template = PromptTemplate.from_template(prompt_two)

**Instantiate Agents**

In [37]:
# Instantiate error handler
class RetryCallbackHandler(BaseCallbackHandler):
    def on_tool_error(
        self, error: Union[Exception, KeyboardInterrupt], **kwargs: Any
    ) -> Any:
        # Log the error or perform any necessary actions
        
        # Retry logic
        max_retries = 1
        current_retry = kwargs.get('retry_count', 0)
        if current_retry < max_retries:
            print(f"Retrying tool execution (Attempt {current_retry + 1})")
            # Increment retry count
            kwargs['retry_count'] = current_retry + 1
            # Re-run the tool
            return AgentAction.RERUN
        else:
            print("Maximum retries reached. Switching to another tool.")
            return AgentAction.CHANGE_TOOL

In [11]:
# The error handler
error_handler = RetryCallbackHandler()

In [30]:
# Create tools
search = TavilySearchResults()

tool_search_item = Tool(
    name="search internet",
    description="Searches the internet for the documentation and examples in order to answer a query",
    func=search.run,
    handle_tool_error=True
)

# List of all tools
tools = [tool_search_item]

In [42]:

# Instantiate agents


agent = create_react_agent(mixtral, tools, prompt_template) 



agent_executor = AgentExecutor(agent=agent,
                                tools=tools,
                                verbose=True,
                                return_intermediate_steps=True,
                                handle_parsing_errors=True,
                                callbacks=[error_handler])


ValueError: Prompt missing required variables: {'tool_names', 'agent_scratchpad'}

**Run the pipeline**

In [39]:
query = "How do I use agents in langchain in order to create a RAG system"

In [40]:
agent_executor.invoke({"input": query})




> Entering new AgentExecutor chain...
To answer this question, I need to find information about using agents in LangChain and how to create a RAG (Red, Amber, Green) system. I will search the internet for relevant documentation and examples.

Action: search internet
Action Input: "langchain agents rag system tutorial"
Observation[{'url': 'https://cheatsheet.md/langchain-tutorials/llm-rag.en', 'content': "This article provides an in-depth understanding of LLM RAG, a vital Language Model in AI, and its working process. We delve into various prompt engineering techniques and their role in enhancing the functionality of LLM RAG. The article also explores practical applications of prompt engineering and its potential to transform LLM RAG's performance."}, {'url': 'https://towardsdatascience.com/using-langchain-react-agents-for-answering-multi-hop-questions-in-rag-systems-893208c1847e', 'content': '[Source]\n--\n--\n2\nWritten by Dr. Varshita Sher\nTowards Data Science\nSenior Data Scienti

{'input': 'How do I use agents in langchain in order to create a RAG system',
 'output': 'To use agents in a LangChain RAG system, follow these steps:\n\n1. Install the necessary libraries:\n\n```python\npip install langchain\npip install openai\npip install faiss-cpu\npip install chromadb\n```\n\n2. Create a retriever to fetch relevant documents based on the user\'s query:\n\n```python\nfrom langchain.vectorstores import Chroma\nfrom langchain.embeddings.openai import OpenAIEmbeddings\nfrom langchain.retrievers import VectorStoreRetriever\n\nembeddings = OpenAIEmbeddings()\nvectorstore = Chroma(embedding_function=embeddings)\nretriever = VectorStoreRetriever(vectorstore=vectorstore)\n```\n\n3. Create a LangChain agent that uses the retrieved documents to generate a response:\n\n```python\nfrom langchain.agents import initialize_agent\nfrom langchain.agents.agent_toolkits import LangchainToolkit\nfrom langchain.llms import OpenAI\n\ntoolkit = LangchainToolkit(retriever=retriever)\nagen